<a href="https://colab.research.google.com/github/Ramubala/face-recognition/blob/main/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
# from google.colab import drive
# drive.mount('/content/drive')

In [101]:
# !rm -rf '/content/data' 
# !rm -rf '/content/annotations'

In [102]:
# !mkdir data
# !mkdir annotations
# !tar zxvf '/content/drive/MyDrive/datasets/originalPics.tar.gz' -C '/content/data/' ;
# !tar zxvf '/content/drive/MyDrive/datasets/FDDB-folds.tgz' -C '/content/annotations/';

In [103]:
# import glob
# file_list = [file for file in glob.glob(r'/content/annotations/FDDB-folds/*-ellipseList.txt',recursive=True)]
# train_files, test_files = file_list[:9],[file_list[9]]
# print(train_files, test_files)

In [104]:
#custom_train_dataset[2]

In [105]:
# def per_person_stats(source_txt: str):
#   stats_dict=dict()
#   with open(source_txt,'r') as f:
#     data = f.readlines()
#     for i in range(1, len(data)):
#       line = data[i].strip().split("\t")
#       stats_dict[line[0]] = int(line[1])
#   f.close()
#   return stats_dict

In [106]:
# def pick_train_pairs(_dict, num_per_sample, num_negative_faces_per_sample):
#   if num_per_sample!=3:
#     raise Exception("only 3 per sample supported by algorithm")
#   allfaces = list(_dict.keys())
#   data = []
#   for key in allfaces:
#     # anchor and postive example
#     facepath = '/content/data/lfw-py/lfw_funneled/{}'.format(key)
#     available_images = [file for file in glob.glob(facepath+'/*.jpg')]
#     num_available_images = _dict[key]
#     num_achor_postive_pairs = num_available_images*(num_available_images-1)
#     if num_achor_postive_pairs==0:
#       combinations = [(available_images[0],available_images[0])]
#     else:
#       combinations = list(itertools.combinations(available_images, 2))

#     # negative face
#     possible_choices = [v for v in allfaces if v != key]
#     negative_examples = random.sample(possible_choices, num_negative_faces_per_sample)
#     for negative_example in negative_examples:
#       negfacepath = '/content/data/lfw-py/lfw_funneled/{}'.format(negative_example)
#       available_images = [file for file in glob.glob(negfacepath+'/*.jpg')]
#       neg_imgpath = random.choice(available_images)

#       for tuple_el in combinations:
#         data.append((tuple_el[0],tuple_el[1],neg_imgpath))
#   return data  

In [107]:
# train_txt_file = '/content/data/lfw-py/peopleDevTrain.txt'
# test_txt_file = '/content/data/lfw-py/peopleDevTest.txt'
# train_stats = per_person_stats(train_txt_file)
# test_stats = per_person_stats(test_txt_file)

In [108]:
#test_stats.keys() & train_stats.keys()

In [109]:
# train_data = pick_train_pairs(train_stats, 3, 5)

In [110]:
import torchvision
import torch
import os
import glob
import numpy as np
import itertools
import random
random.seed(42)
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms

In [111]:
train_dataset = torchvision.datasets.LFWPeople('/content/data/',split='train',download=True)
test_dataset = torchvision.datasets.LFWPeople('/content/data/',split='test',download=True)

Files already downloaded and verified
Files already downloaded and verified


In [112]:
len(train_dataset), len(test_dataset)

(9525, 3708)

In [113]:
train_dataset[0][0], train_dataset[0][1]

(<PIL.Image.Image image mode=RGB size=250x250 at 0x7FBB8F2C64D0>, 0)

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, src_dataset, upsampling_factor):
    self.src_dataset = src_dataset
    self.train_labels = np.array([src_dataset[i][1] for i in range(len(src_dataset))])
    self.labels = list(set(self.train_labels))
    self.label_to_index= {label: np.where(self.train_labels == label)[0]
                                     for label in self.labels}
    self.upsampling_factor = upsampling_factor
    self.transform = transforms.Compose([
          transforms.ToTensor()
          ])
    
  def __len__(self):
      return self.upsampling_factor*len(self.src_dataset)

  def __getitem__(self, idx):
      # anchor image
      ref_dataset_index = int(idx/self.upsampling_factor)
      ref_train_image = self.src_dataset[ref_dataset_index]#[0], self.src_dataset[ref_dataset_index][1]
      label = ref_train_image[1]

      # finding positive example
      possible_ref_choices = self.label_to_index[label]
      choice = random.choice(possible_ref_choices)
      positive_train_image = self.src_dataset[choice]#[0], self.src_dataset[choice][1]

      # finding negative example
      possible_neg_choices = [v for v in self.labels if v!= label]
      choice = random.choice(possible_neg_choices)
      negative_train_image = self.src_dataset[choice]#[0], self.src_dataset[choice][1]

      return (self.transform(ref_train_image[0]), self.transform(positive_train_image[0]), self.transform(negative_train_image[0])), (ref_train_image[1], positive_train_image[1], negative_train_image[1])


_train_dataset = CustomDataset(train_dataset, 1)
_test_dataset = CustomDataset(test_dataset, 1)

In [ ]:
len(_train_dataset), len(_test_dataset)

In [ ]:
_train_dataset[0][0][0].shape

In [ ]:
# torch.save(_train_dataset, 'train.pt')
# torch.save(_test_dataset,'test.pt')

In [ ]:
train_dataloader = DataLoader(_train_dataset,batch_size=32)
test_dataloader = DataLoader(_test_dataset, batch_size=32)

## Define models

In [ ]:
import torch
import torch.nn as nn
from torch.nn.modules.pooling import MaxPool2d

In [ ]:
class BasicConv2d(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, stride ):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,stride=stride),
        nn.BatchNorm2d(num_features=out_channels)
    )
    self.relu = nn.ReLU()

  def forward(self,x):
    return self.relu(self.model(x))

In [ ]:
class FaceSimilarity(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv_layers = nn.Sequential(nn.Conv2d(3, 32, 5), 
                                     nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(32, 64, 5), 
                                     nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2))

    self.fc_layers = nn.Sequential(nn.Linear(64 * 59 * 59, 256),
                            nn.PReLU(),
                            nn.Linear(256, 256),
                            nn.PReLU(),
                            nn.Linear(256, 128)
                            )
    # self.conv_layers = nn.Sequential(
    #     BasicConv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2),
    #     nn.MaxPool2d(kernel_size=3,stride=2),
    #     #nn.LayerNorm(55),
    #     BasicConv2d(in_channels=64,out_channels=64,kernel_size=1,stride=1),
    #     BasicConv2d(in_channels=64,out_channels=192,kernel_size=3,stride=1),
    #     #nn.LayerNorm(55),
    #     nn.MaxPool2d(kernel_size=3,stride=2),
    #     BasicConv2d(in_channels=192,out_channels=192,kernel_size=1,stride=1),
    #     BasicConv2d(in_channels=192,out_channels=384,kernel_size=3,stride=1),
    #     nn.MaxPool2d(kernel_size=3,stride=2),
    #     BasicConv2d(in_channels=384,out_channels=384,kernel_size=1,stride=1),
    #     BasicConv2d(in_channels=384,out_channels=256,kernel_size=3,stride=1),
    #     BasicConv2d(in_channels=256,out_channels=256,kernel_size=1,stride=1),
    #     BasicConv2d(in_channels=256,out_channels=256,kernel_size=3,stride=1),
    #     BasicConv2d(in_channels=256,out_channels=256,kernel_size=1,stride=1),
    #     BasicConv2d(in_channels=256,out_channels=256,kernel_size=3,stride=1),
    #     nn.MaxPool2d(kernel_size=3,stride=2),
    # )

    # self.fc_layers = nn.Sequential(
    #     nn.Linear(in_features=2*2*256,out_features=128),
    #     nn.Linear(in_features=128, out_features=128),
    #     #nn.Linear(in_features=32*128, out_features=1*128)
    # )

  def forward(self,x):
    conv_output = self.conv_layers(x)
    conv_output = conv_output.view(conv_output.size()[0], -1)
    linear_layers_output  = self.fc_layers(conv_output)
    l2_norm_output  = linear_layers_output/(linear_layers_output.pow(2).sum(1, keepdim=True).sqrt())
    return l2_norm_output

In [ ]:
class TripletModel(nn.Module):
    def __init__(self):
      super().__init__()
      self.face_similarity = FaceSimilarity()
    
    def forward(self, anchor, positive, negative):
      rep_1 = self.face_similarity(anchor).reshape(-1,128)
      rep_2 = self.face_similarity(positive).reshape(-1,128)
      rep_3 = self.face_similarity(negative).reshape(-1,128)
      return rep_1, rep_2, rep_3

Define metrics

In [ ]:
class TripletLoss(nn.Module):
  def __init__(self, alpha):
    super().__init__()
    self.margin = alpha

  def forward(self, anchor, positive, negative):
    distance_positive = (anchor - positive).pow(2).sum(1)
    distance_negative = (anchor - negative).pow(2).sum(1)
    losses = torch.nn.functional.relu(distance_positive - distance_negative + self.margin)
    return losses.mean()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = TripletLoss(alpha=1.2)
model = TripletModel().to(device)
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def train_model(dataloader,model,optimizer,criterion):
  model.train()
  train_loss=0
  for i, (input,target) in enumerate(dataloader):
    optimizer.zero_grad()
    output = model(input[0], input[1], input[2])
    loss = criterion(output[0],output[1],output[2])
    loss.backward()
    train_loss+=loss
    optimizer.step()
  return train_loss/len(dataloader)

In [ ]:
def eval_model(dataloader,model,optimizer,criterion):
  model.eval()
  val_loss=0
  classification_accuracy=0
  with optimizer.zero_grad():
    for i, (input,target) in enumerate(dataloader):
      output = model(input[0],input[1],input[2])
      loss = criterion(output[0],output[1],output[2])
      classification_accuracy+=torch.count.non_zero(loss)/len(loss)
      #loss.backward()
      val_loss+=loss
    return val_loss/len(dataloader), classification_accuracy/len(dataloader)

In [ ]:
def fit(max_epochs, model, optimizer, criterion, lr):
  min_val_loss = float('inf')

  for i in range(max_epochs):
    # train model
    train_loss, train_accuracy = train_model(train_dataloader,model, optimizer, criterion)
    
    # val model loss
    val_loss, val_accuracy = eval_model(test_dataloader, model, optimizer, criterion)

    print("Epoch {} elapsed, train_accuracy: {}, val_accuracy: {} , train_loss: {}, val_loss: {}".format(train_accuracy,val_accuracy,train_loss,val_loss))
    if val_loss<min_val_loss:
      torch.save('saved_model/face_verification.pt')

In [ ]:
fit(1, model, optimizer, criterion, 0.1)

In [ ]:
saved_model = TripletModel()
saved_model.load_state_dict(torch.load('saved_model/face_verification.pt'))

In [ ]:
eval_model(test_dataloader)